In [1]:
#import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [2]:
teamPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/teamData.csv'
playerPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/playerData.csv'
outcomePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/outcomeData.csv'
teamData = pd.read_csv(teamPath)
playerData = pd.read_csv(playerPath, engine='python')
outcomeData = pd.read_csv(outcomePath)

In [3]:
outcomeData = outcomeData.drop(["Start Time", "Box Score", "Notes"], axis=1)


,Year,Month,Visitor,VisitorPTS,Home,HomePTS,OT?,Attend,Total Points,Winner
0,2005,november,Houston Rockets,79,Detroit Pistons,87,NaN,22076,166,Detroit Pistons
1,2005,november,Sacramento Kings,98,Dallas Mavericks,107,NaN,20041,205,Dallas Mavericks
2,2005,november,Denver Nuggets,78,Los Angeles Lakers,89,NaN,18997,167,Los Angeles Lakers
3,2005,november,Indiana Pacers,109,Cleveland Cavaliers,104,2OT,19730,213,Indiana Pacers
4,2005,november,Milwaukee Bucks,92,Orlando Magic,93,NaN,15138,185,Orlando Magic


In [4]:
outcomeData.loc[(outcomeData["OT?"].isna()),'OT?']= 0
outcomeData.loc[(outcomeData["OT?"] == "OT"),'OT?']= 1
outcomeData.loc[(outcomeData["OT?"] == "2OT"),'OT?']= 2
outcomeData.loc[(outcomeData["OT?"] == "3OT"),'OT?']= 3
outcomeData.loc[(outcomeData["OT?"] == "4OT"),'OT?']= 4

In [5]:
def redefineTeamNames(dataframe):
    dataframe = dataframe.replace('Seattle SuperSonics', 'Oklahoma City Thunder', regex=True)
    
    dataframe = dataframe.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans', regex=True)

    dataframe = dataframe.replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True)
    
    dataframe = dataframe.replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True)
    
    dataframe = dataframe.replace('New Jersey Nets', 'Brooklyn Nets', regex=True)
    
#     dataframe = dataframe.replace('*', '', regex=True)

    return dataframe

outcomeData = redefineTeamNames(outcomeData)

In [6]:
def homeWin(dataframe):
    if dataframe['Winner'] == dataframe['Home']:
        return True
    else:
        return False

outcomeData['HomeWin'] = outcomeData.apply(homeWin, axis=1)

In [7]:
encoding = LabelEncoder()
encoding.fit(outcomeData['Home'].values)
outcomeData['Home'] = encoding.transform(outcomeData['Home'].values)
outcomeData['Visitor'] = encoding.transform(outcomeData['Visitor'].values)

In [17]:
featurePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/DifferentialFeatureData.csv'
X = pd.read_csv(featurePath)

train_stats = X.describe()
train_stats = train_stats.transpose()
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
X = norm(X)


y = outcomeData["HomeWin"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
validation_data = (X_test, y_test)

In [18]:
from keras import Sequential
from keras.layers import Dense, Dropout

classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal', input_dim=113))
classifier.add(Dropout(.5))
#Second  Hidden Layer
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(.5))
#Third  Hidden Layer
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dropout(.5))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [19]:
from keras.callbacks import EarlyStopping
#Fitting the data to the training dataset
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
classifier.fit(X_train,y_train, validation_data = validation_data, batch_size=128, epochs=100, callbacks=[es])

Train on 14303 samples, validate on 6131 samples
Epoch 1/100
14303/14303 [==============================] - 3s 239us/step - loss: 0.6228 - accuracy: 0.6401 - val_loss: 0.5853 - val_accuracy: 0.6836
Epoch 2/100
14303/14303 [==============================] - 1s 95us/step - loss: 0.5965 - accuracy: 0.6879 - val_loss: 0.5853 - val_accuracy: 0.6836
Epoch 3/100
14303/14303 [==============================] - 1s 94us/step - loss: 0.5935 - accuracy: 0.6883 - val_loss: 0.5829 - val_accuracy: 0.6904
Epoch 4/100
14303/14303 [==============================] - 2s 121us/step - loss: 0.5950 - accuracy: 0.6911 - val_loss: 0.5823 - val_accuracy: 0.6901
Epoch 5/100
14303/14303 [==============================] - 2s 138us/step - loss: 0.5911 - accuracy: 0.6915 - val_loss: 0.5828 - val_accuracy: 0.6899
Epoch 6/100
14303/14303 [==============================] - 1s 100us/step - loss: 0.5891 - accuracy: 0.6913 - val_loss: 0.5820 - val_accuracy: 0.6912
Epoch 7/100
 2432/14303 [====>.........................] - 

/Users/lorneez/opt/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.223290). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)
/Users/lorneez/opt/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.113132). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


14303/14303 [==============================] - 2s 131us/step - loss: 0.5890 - accuracy: 0.6945 - val_loss: 0.5817 - val_accuracy: 0.6893
Epoch 8/100
14303/14303 [==============================] - 1s 104us/step - loss: 0.5886 - accuracy: 0.6932 - val_loss: 0.5813 - val_accuracy: 0.6898
Epoch 9/100
14303/14303 [==============================] - 2s 114us/step - loss: 0.5866 - accuracy: 0.6966 - val_loss: 0.5808 - val_accuracy: 0.6924
Epoch 10/100
14303/14303 [==============================] - 1s 82us/step - loss: 0.5866 - accuracy: 0.6935 - val_loss: 0.5839 - val_accuracy: 0.6893
Epoch 11/100
14303/14303 [==============================] - 1s 95us/step - loss: 0.5875 - accuracy: 0.6950 - val_loss: 0.5828 - val_accuracy: 0.6890
Epoch 12/100
14303/14303 [==============================] - 2s 118us/step - loss: 0.5873 - accuracy: 0.6942 - val_loss: 0.5821 - val_accuracy: 0.6891
Epoch 13/100
14303/14303 [==============================] - 2s 165us/step - loss: 0.5887 - accuracy: 0.6928 - val_los

In [20]:
#eval_model=classifier.evaluate(X_train, y_train)
#eval_model
eval_model=classifier.evaluate(X_test, y_test)
eval_model

6131/6131 [==============================] - 1s 92us/step


[0.5818697779111733, 0.6912412047386169]

In [12]:
from sklearn.metrics import confusion_matrix

y_pred=classifier.predict(X_test)
y_pred =(y_pred> 0.6)


In [13]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1808  686]
 [1306 2331]]
